Problem statement
Imagine you're developing a BlogPost Aggregator application that gathers and organizes blog post data from multiple sources using APIs. Your goal is to fetch and display blog post information, including titles and authors, in an organized manner.

Exercise
Your application needs to perform the following tasks:

1. Fetch Users: Retrieve a list of users from the API.
2. Fetch Posts: Fetch a list of blog posts from the API.
3. Fetch Comments: For each post, fetch the associated comments.
4. Organize Data: Organize the fetched data to display blog post titles, authors, and the number of comments each post has.change column name id to userId in users,id to postId in posts and id to commentId,name to commented_by_name, email to commented_by_email, body to comment_body in comments. combine the data

In [ ]:
import pandas as pd
import requests

# Define API endpoints
link_mapping = {
    "users": "https://jsonplaceholder.typicode.com/users",
    "posts": "https://jsonplaceholder.typicode.com/posts",
    "comments": "https://jsonplaceholder.typicode.com/comments"
}

# Function to fetch data from API and convert it into a DataFrame with specific column names
def fetch_data(link: str, rename_map: dict) -> pd.DataFrame:
    response = requests.get(link)
    if response.status_code == 200:
        print(f'Get Response for link {link} is {response.status_code}')
        data = response.json()
        df = pd.DataFrame(data)
        df.rename(columns=rename_map, inplace=True)
        return df
    else:
        print(f'Failed to get data from {link}')
        return pd.DataFrame()

# Rename mappings for each API response
rename_mappings = {
    "users": {
        "id": "userId"
    },
    "posts": {
        "id": "postId",
        "userId": "userId"
    },
    "comments": {
        "id": "commentId",
        "name": "commented_by_name",
        "email": "commented_by_email",
        "body": "comment_body",
        "postId": "postId"
    }
}

# Fetch data and store in DataFrames
df_mapping = {}
for key in link_mapping:
    df_mapping[key] = fetch_data(link_mapping[key], rename_mappings[key])

# Merge the DataFrames
# Merge users with posts on 'userId'
merged_data = pd.merge(df_mapping['posts'], df_mapping['users'], on='userId', how='left')

# Merge the result with comments on 'postId'
final_data = pd.merge(merged_data, df_mapping['comments'], on='postId', how='left')

# Group by post title and count the number of comments
organized_data = final_data.groupby(['postId', 'title', 'userId', 'name']).agg(num_comments=('commentId', 'count'))

# Display the organized DataFrame
print(organized_data)


Get Response for link https://jsonplaceholder.typicode.com/users is 200
Get Response for link https://jsonplaceholder.typicode.com/posts is 200
Get Response for link https://jsonplaceholder.typicode.com/comments is 200
                                                                                     num_comments
postId title                                              userId name                            
1      sunt aut facere repellat provident occaecati ex... 1      Leanne Graham                  5
2      qui est esse                                       1      Leanne Graham                  5
3      ea molestias quasi exercitationem repellat qui ... 1      Leanne Graham                  5
4      eum et est occaecati                               1      Leanne Graham                  5
5      nesciunt quas odio                                 1      Leanne Graham                  5
...                                                                                           .